In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

# Data Pre-Processing



Data pre-processing took the following path:

Data Cleaning

The follow procedures were applied in data cleaning to address missing values and outliers.

- Athlete Names: Data with missing values or erroneous entries in names of athletes was corrected
- Penalty: Rows consisting of a penalty greater than 1 were deleted
- Score: Rows consisting of in either E_Score or D_Score were deleted
- Round Names: some scores were recorded repeatedly due to different round names, so round names were organized.
- Apparatus Names(VT, HB): uniform terminology to facilitate smoother analysis
- Addition of full name column(ex: SIMONE Biles)

Data Pre-Processing

Data pre-processing took the following path:

Data Cleaning

The follow procedures were applied in data cleaning to address missing values and outliers.

- Athlete Names: Data with missing values or erroneous entries in names of athletes was corrected
- Penalty: Rows consisting of a penalty greater than 1 were deleted
- Score: Rows consisting of in either E_Score or D_Score were deleted
- Round Names: some scores were recorded repeatedly due to different round names, so round names were organized.
- Apparatus Names(VT, HB): uniform terminology to facilitate smoother analysis
- Addition of full name column(ex: SIMONE Biles)

In [2]:
df1 = pd.read_csv('data_2017_2021.csv')
df2 = pd.read_csv('data_2022_2023.csv')

In [3]:
#Name modification

df2.loc[(df2["FirstName"]== "Kate") & (df2["Competition"]=='2022 51st FIG Artistic Gymnastics World Championships'),["LastName"]] = "MCDONALD"
df2.loc[(df2["FirstName"]== "Kate") & (df2["Competition"]=='BIRMINGHAM 2022 Commonwealth Games'),["LastName"]] = "MCDONALD"
df2.loc[(df2["FirstName"]== "Daria Angelina") & (df2["Competition"]=='2023 Varna World Challenge Cup Results'),["LastName"]] = "HARTMANN"
df2.loc[(df2["LastName"]== "ABIYURAFI") & (df2["Competition"]=='FIG Apparatus World Cup 2023'),["FirstName"]] = "ABIYURAFI"
df2.loc[(df2["LastName"]== "CALVO MORENO JO") & (df2["Competition"]=='2022 51st FIG Artistic Gymnastics World Championships'),["FirstName"]] = "Jossimar Orlando"
df2.loc[(df2["FirstName"]== "Jossimar Orlando") & (df2["Competition"]=='2022 51st FIG Artistic Gymnastics World Championships'),["LastName"]] = "CALVO MORENO"
df2.loc[(df2["LastName"]== "ELPITIYA BADALG D") & (df2["Competition"]=='BIRMINGHAM 2022 Commonwealth Games'),["FirstName"]] = "G.D."
df2.loc[(df2["FirstName"]== "G.D.") & (df2["Competition"]=='BIRMINGHAM 2022 Commonwealth Games'),["LastName"]] = "ELPITIYA BADALG"
df2.loc[(df2["LastName"]== "KHOSRONEZHAD") & (df2["Competition"]=='FIG Apparatus World Cup 2023'),["FirstName"]] = "Mohammadreza"
df2.loc[(df2["LastName"]== "KUMARASINGHEGE HG") & (df2["Competition"]=='BIRMINGHAM 2022 Commonwealth Games'),["FirstName"]] = "HG"
df2.loc[(df2["FirstName"]== "HG") & (df2["Competition"]=='BIRMINGHAM 2022 Commonwealth Games'),["LastName"]] = "KUMARASINGHEGE"
df2.loc[(df2["LastName"]== "NOURISHOURAKCHALI") & (df2["Competition"]=='FIG Apparatus World Cup 2023'),["FirstName"]] = "Hossein"
df2.loc[(df2["LastName"]== "YONEKURA") & (df2["Competition"]=='FIG Apparatus World Cup 2023'),["FirstName"]] = "Hidenobu"

In [4]:
df1.loc[(df1["LastName"]== "JESUS SANT") & (df1["Competition"]=='Olympic Games'),["FirstName"]] = "Melanie"
df1.loc[(df1["FirstName"]== "Melanie") & (df1["Competition"]=='Olympic Games'),["LastName"]] = "JESUS DOS SANTOS"

In [5]:
# Combine data

data = pd.concat([df1, df2], ignore_index=True)

In [54]:
data.to_csv("gymnastics.csv")

# Weighted Mean



- In order to reflect the recent form, we calculated the weighted average of D and E Scores for each athlete and apparatus, giving more weight to competitions closer to the Paris Olympics.

In [6]:

# Convert date data to datetime format
data['Date'] = data['Date'].apply(lambda x: ' '.join(x.split()[::-1]))
data['Date'] = data['Date'].str.replace(r'-.*', '', regex=True)
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')


# data preprocessing (handling missing values)
data_cleaned = data.dropna(subset=['Date', 'FirstName', 'Country', 'Rank', 'D_Score', 'E_Score', 'Score']).reset_index(drop=True)
data_cleaned['Penalty'].fillna(0, inplace=True)


data_cleaned= data_cleaned.replace({'Apparatus': {'VT1': 'VT', 'VT2': 'VT', 'UE' : 'UB','hb' : 'HB'}})
rounds_to_combine = ['qual', 'AAqual', 'TeamQual']
data_cleaned['Round'] = data_cleaned['Round'].replace(rounds_to_combine, 'qual')

# Treat rows with the same 'Round', 'Apparatus', 'Score' for each competition and player as duplicates and keep only one
data_cleaned = data_cleaned.drop_duplicates(subset=['Competition', 'LastName', 'FirstName', 'Round', 'Apparatus']).reset_index(drop=True) #Score를 뺐음 (아마도 소숫점 때문에 오차가 나서 중복항 삭제가 잘 안 되는 듯)

# Combine LastName and FirstName for player identification
data_cleaned['Athlete'] = data_cleaned['LastName'] + " " + data_cleaned['FirstName']

# Create Home Advantage variable
country_df = pd.read_excel("country_abb.xls")
country_abb = dict(zip(country_df['ABB'], country_df['Country']))
country_abb["nan"] = ""
data_cleaned["Location_Country"] = ""
data_cleaned["Country_Country"] = ""
for i in range(0, data_cleaned.shape[0]):
    if len(data_cleaned.at[i, "Location"].split(",")) > 1:
        data_cleaned.at[i, "Location_Country"] = data_cleaned.at[i, "Location"].split(",")[1].lstrip()
    else:
        data_cleaned.at[i, "Location_Country"] = data_cleaned.at[i, "Location"].strip()
    data_cleaned.at[i, "Country_Country"] = country_abb[str(data_cleaned.at[i, "Country"])]
data_cleaned["Home_advantage"] = (data_cleaned["Location_Country"] == data_cleaned["Country_Country"]).astype(int)

# Create id variable country + fullname
data_cleaned["id"] = ""

for i in range(data_cleaned.shape[0]):
   if pd.notna(data_cleaned.at[i, "LastName"]):  # "LastName"이 NaN이 아닌 경우
        data_cleaned.at[i, "full_name"] = str(data_cleaned.at[i, "FirstName"]).lower() + "_" + str(data_cleaned.at[i, "LastName"]).lower()
   else: # if "LastName" is NaN
       data_cleaned.at[i, "full_name"] = str(data_cleaned.at[i, "FirstName"]).lower()

for i in range(0, data_cleaned.shape[0]):
    data_cleaned.at[i, "id"] = data_cleaned.at[i, "Country_Country"] + "_" + data_cleaned.at[i, "full_name"] #+ "_" + data_cleaned.at[i, "Apparatus"]

# Combine ridge regression coefficient variables
ridge = pd.read_csv("betas_whole.csv")
data_cleaned = pd.merge(data_cleaned, ridge, on='id', how='left')

# Sort data based on date
data_cleaned.sort_values(by='Date', inplace=True)

# Set time decay rate
lambda_decay = 0.01


C:\Users\sbjbo\AppData\Local\Temp\ipykernel_37872\1580079411.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Date'] = pd.to_datetime(data['Date'], errors='coerce')


In [7]:
# First sort by 'Score' in the data_cleaned dataframe, then by 'D_Score' (in order of higher D_Score within the same Score)
data_cleaned = data_cleaned.sort_values(by=['Score', 'D_Score'], ascending=[False, False])

# Rank the sorted data (applied to all Rounds)
data_cleaned['Adjusted_Rank'] = data_cleaned.groupby(['Competition', 'Round', 'Apparatus']).cumcount() + 1

# Check the sorted and ranked results
data_cleaned.head()

LastName FirstName Gender Country       Date  \
19445      ZOU  Jingyuan      m     CHN 2022-11-06   
19449      ZOU  Jingyuan      m     CHN 2023-08-01   
19456      ZOU  Jingyuan      m     CHN 2023-09-24   
19455      ZOU  Jingyuan      m     CHN 2023-09-24   
19450      ZOU  Jingyuan      m     CHN 2023-08-01   

                                             Competition  Round  \
19445  2022 51st FIG Artistic Gymnastics World Champi...  final   
19449                   2023 FISU World University Games  final   
19456                     HANGZHOU 2022 19th Asian Games   qual   
19455                     HANGZHOU 2022 19th Asian Games  final   
19450                   2023 FISU World University Games   qual   

                 Location Apparatus  Rank  ...  Penalty   Score       Athlete  \
19445  Liverpool, England        PB   1.0  ...      0.0  16.166  ZOU Jingyuan   
19449      Chengdu, China        PB   1.0  ...      0.0  16.066  ZOU Jingyuan   
19456     Hangzhou, China        PB   1.0  ...      0.0  15.933  ZOU Jingyuan   
19455     Hangzhou, China        PB   1.0  ...      0.0  15.933  ZOU Jingyuan   
19450      Chengdu, China        PB   1.0  ...      0.0  15.866  ZOU Jingyuan   

       Location_Country Country_Country Home_advantage                  id  \
19445           England           China              0  China_jingyuan_zou   
19449             China           China              1  China_jingyuan_zou   
19456             China           China              1  China_jingyuan_zou   
19455             China           China              1  China_jingyuan_zou   
19450             China           China              1  China_jingyuan_zou   

          full_name      Beta Adjusted_Rank  
19445  jingyuan_zou  0.008972             1  
19449  jingyuan_zou  0.008972             1  
19456  jingyuan_zou  0.008972             1  
19455  jingyuan_zou  0.008972             1  
19450  jingyuan_zou  0.008972             1  

[5 rows x 22 columns]

In [8]:
# sort by 'D_Score' in the data_cleaned dataframe
data_cleaned = data_cleaned.sort_values(by='D_Score', ascending=False)

# Rank the sorted data (applied to all Rounds)
data_cleaned['Adjusted_DRank'] = data_cleaned.groupby(['Competition', 'Round', 'Apparatus']).cumcount() + 1

# Check the sorted and ranked results
data_cleaned.head()

LastName FirstName Gender Country       Date  \
19445       ZOU  Jingyuan      m     CHN 2022-11-06   
19449       ZOU  Jingyuan      m     CHN 2023-08-01   
18576  WHITLOCK       Max      m     GBR 2023-10-08   
162        GUAN  Chenchen      w     CHN 2021-07-25   
13767  PHILLIPS    Curran      m     USA 2023-08-04   

                                             Competition    Round  \
19445  2022 51st FIG Artistic Gymnastics World Champi...    final   
19449                   2023 FISU World University Games    final   
18576  2023 52nd FIG Artistic Gymnastics World Champi...    final   
162                                        Olympic Games     qual   
13767                        2023 Core Hydration Classic  AAfinal   

                        Location Apparatus  Rank  ...   Score  \
19445         Liverpool, England        PB   1.0  ...  16.166   
19449             Chengdu, China        PB   1.0  ...  16.066   
18576           Antwerp, Belgium        PH   5.0  ...  14.300   
162                 Tokyo, Japan        BB   1.0  ...  14.933   
13767  Hoffman Estates, Illinois        PB   1.0  ...  14.750   

               Athlete  Location_Country           Country_Country  \
19445     ZOU Jingyuan           England                     China   
19449     ZOU Jingyuan             China                     China   
18576     WHITLOCK Max           Belgium             Great Britain   
162      GUAN Chenchen             Japan                     China   
13767  PHILLIPS Curran          Illinois  United States of America   

      Home_advantage                                        id  \
19445              0                        China_jingyuan_zou   
19449              1                        China_jingyuan_zou   
18576              0                Great Britain_max_whitlock   
162                0                       China_chenchen_guan   
13767              0  United States of America_curran_phillips   

             full_name      Beta Adjusted_Rank Adjusted_DRank  
19445     jingyuan_zou  0.008972             1              1  
19449     jingyuan_zou  0.008972             1              1  
18576     max_whitlock  0.004215             5              1  
162      chenchen_guan -0.052936             1              1  
13767  curran_phillips  0.013437             4              1  

[5 rows x 23 columns]

In [9]:
# sort by 'E_Score' in the data_cleaned dataframe
data_cleaned = data_cleaned.sort_values(by='E_Score', ascending=False)

# Rank the sorted data (applied to all Rounds)
data_cleaned['Adjusted_ERank'] = data_cleaned.groupby(['Competition', 'Round', 'Apparatus']).cumcount() + 1

# Check the sorted and ranked results
data_cleaned.head()

LastName FirstName Gender Country       Date  \
1132   ANDRADE    Rebeca      w     BRA 2023-10-21   
12692   NGUYEN   Brandon      m     USA 2023-08-04   
1479      ASIL      Adem      m     TUR 2023-03-17   
1364    ARTLIP   Michael      m     USA 2023-08-04   
6314      HALE    Dallas      m     USA 2023-08-04   

                                Competition    Round  \
1132   SANTIAGO 2023 XIX Pan American Games    final   
12692           2023 Core Hydration Classic  AAfinal   
1479     EnBW DTB Pokal Team Challenge 2023     qual   
1364            2023 Core Hydration Classic  AAfinal   
6314            2023 Core Hydration Classic  AAfinal   

                        Location Apparatus  Rank  ...         Athlete  \
1132             Santiago, Chile        VT   1.0  ...  ANDRADE Rebeca   
12692  Hoffman Estates, Illinois        VT   8.0  ...  NGUYEN Brandon   
1479          Stuttgart, Germany        VT   3.0  ...       ASIL Adem   
1364   Hoffman Estates, Illinois        VT   9.0  ...  ARTLIP Michael   
6314   Hoffman Estates, Illinois        VT   4.0  ...     HALE Dallas   

       Location_Country           Country_Country  Home_advantage  \
1132              Chile                    Brazil               0   
12692          Illinois  United States of America               0   
1479            Germany                    Turkey               0   
1364           Illinois  United States of America               0   
6314           Illinois  United States of America               0   

                                            id       full_name      Beta  \
1132                     Brazil_rebeca_andrade  rebeca_andrade  0.004546   
12692  United States of America_brandon_nguyen  brandon_nguyen -0.001349   
1479                          Turkey_adem_asil       adem_asil -0.001372   
1364   United States of America_michael_artlip  michael_artlip  0.013278   
6314      United States of America_dallas_hale     dallas_hale  0.029169   

       Adjusted_Rank Adjusted_DRank Adjusted_ERank  
1132               1              1              1  
12692              8             46              1  
1479               2              2              1  
1364              11             52              2  
6314               4             17              3  

[5 rows x 24 columns]

In [10]:
def calculate_rank_contribution_v2(group):
    # Sort the group by Score in descending order (higher score is better)
    sorted_group = group.sort_values(by='Score', ascending=False).reset_index()

    # Initialize a dictionary to store the rank contributions
    rank_contributions = {idx: 0 for idx in sorted_group.index}

    # Calculate the rank contributions
    for i in range(len(sorted_group)):
        for j in range(i + 1, len(sorted_group)):
            score_diff = sorted_group.at[i, 'Score'] - sorted_group.at[j, 'Score']
            rank_diff = j - i
            rank_contributions[i] += score_diff / rank_diff

    # Assign the calculated contributions back to the original index
    sorted_group['RankContribution'] = sorted_group.index.map(rank_contributions)
    return sorted_group.set_index('index')

# Apply the updated function to each group of Competition and Apparatus
rank_contributions_v2 = data_cleaned.groupby(['Competition', 'Apparatus']).apply(calculate_rank_contribution_v2)

# Drop the level added by groupby and sort by the original index
rank_contributions_v2.reset_index(level=[0, 1], drop=True, inplace=True)
rank_contributions_v2.sort_index(inplace=True)

# Update the RankContribution column in the original data
data_cleaned['RankContribution'] = rank_contributions_v2['RankContribution']

# Recalculate the total rank contribution for each athlete
data_cleaned['TotalRankContribution'] = data_cleaned.groupby(['FirstName', 'LastName','Apparatus'])['RankContribution'].transform('sum')

# Display the updated data
data_cleaned.head()


LastName FirstName Gender Country       Date  \
1132   ANDRADE    Rebeca      w     BRA 2023-10-21   
12692   NGUYEN   Brandon      m     USA 2023-08-04   
1479      ASIL      Adem      m     TUR 2023-03-17   
1364    ARTLIP   Michael      m     USA 2023-08-04   
6314      HALE    Dallas      m     USA 2023-08-04   

                                Competition    Round  \
1132   SANTIAGO 2023 XIX Pan American Games    final   
12692           2023 Core Hydration Classic  AAfinal   
1479     EnBW DTB Pokal Team Challenge 2023     qual   
1364            2023 Core Hydration Classic  AAfinal   
6314            2023 Core Hydration Classic  AAfinal   

                        Location Apparatus  Rank  ...  \
1132             Santiago, Chile        VT   1.0  ...   
12692  Hoffman Estates, Illinois        VT   8.0  ...   
1479          Stuttgart, Germany        VT   3.0  ...   
1364   Hoffman Estates, Illinois        VT   9.0  ...   
6314   Hoffman Estates, Illinois        VT   4.0  ...   

                Country_Country  Home_advantage  \
1132                     Brazil               0   
12692  United States of America               0   
1479                     Turkey               0   
1364   United States of America               0   
6314   United States of America               0   

                                            id       full_name      Beta  \
1132                     Brazil_rebeca_andrade  rebeca_andrade  0.004546   
12692  United States of America_brandon_nguyen  brandon_nguyen -0.001349   
1479                          Turkey_adem_asil       adem_asil -0.001372   
1364   United States of America_michael_artlip  michael_artlip  0.013278   
6314      United States of America_dallas_hale     dallas_hale  0.029169   

      Adjusted_Rank Adjusted_DRank  Adjusted_ERank RankContribution  \
1132              1              1               1         5.969039   
12692             8             46               1         1.936200   
1479              2              2               1         5.082962   
1364             11             52               2         2.101933   
6314              4             17               3         2.942574   

      TotalRankContribution  
1132              37.926920  
12692              1.936200  
1479              65.493201  
1364               4.540594  
6314               2.942574  

[5 rows x 26 columns]

In [11]:
data_cleaned["Competition_Round"] = data_cleaned["Competition"] + data_cleaned["Round"]


# function for calculating weighted average of D_Score and E_Score
def calculate_weighted_average_d_e(group, score_type):
    weighted_averages = {}

    for index, row in group.iterrows():
      prediction_date = pd.Timestamp(row["Date"])
      if group.shape[0] > 1:
        group_deleted = group.drop(index)
        #continue
      else:
        group_deleted = group

      # calculate differences between prediction dates and each competition date(in days)
      time_diff = (prediction_date - group_deleted['Date']).dt.days

      # time decay weights
      weights = np.exp(-lambda_decay * abs(time_diff))
    

      # weighted average
      weighted_average = np.average(group_deleted[score_type], weights=weights)

      weighted_averages[row["Competition_Round"]] = weighted_average #Competition대신 해당 행을 나타낼 수 있는 것 필요

    return weighted_averages

# calculate weighted average of D_Score and E_Score
weighted_averages_d = data_cleaned.groupby(['Athlete','Apparatus']).apply(lambda x: calculate_weighted_average_d_e(x, 'D_Score'))
weighted_averages_e = data_cleaned.groupby(['Athlete','Apparatus']).apply(lambda x: calculate_weighted_average_d_e(x, 'E_Score'))
weighted_averages_rank = data_cleaned.groupby(['Athlete','Apparatus']).apply(lambda x: calculate_weighted_average_d_e(x, 'Adjusted_Rank'))
weighted_averages_Drank = data_cleaned.groupby(['Athlete','Apparatus']).apply(lambda x: calculate_weighted_average_d_e(x, 'Adjusted_DRank'))
weighted_averages_Erank = data_cleaned.groupby(['Athlete','Apparatus']).apply(lambda x: calculate_weighted_average_d_e(x, 'Adjusted_ERank'))
weighted_averages_Rankcontr = data_cleaned.groupby(['Athlete','Apparatus']).apply(lambda x: calculate_weighted_average_d_e(x, 'RankContribution'))
weighted_averages_d.head(), weighted_averages_e.head()


(Athlete      Apparatus
 AAS Fredrik  FX               {'2023 FISU World University Gamesqual': 4.0}
              HB           {'2022 Cottbus World Cupqual': 4.6, '2023 FISU...
              PB           {'2023 FISU World University Gamesqual': 3.9, ...
              PH           {'2023 Cottbus World Cupqual': 4.3771895237440...
              SR               {'2023 FISU World University Gamesqual': 3.5}
 dtype: object,
 Athlete      Apparatus
 AAS Fredrik  FX             {'2023 FISU World University Gamesqual': 8.566}
              HB           {'2022 Cottbus World Cupqual': 6.9483167929052...
              PB           {'2023 FISU World University Gamesqual': 6.900...
              PH           {'2023 Cottbus World Cupqual': 7.1975685712321...
              SR             {'2023 FISU World University Gamesqual': 8.366}
 dtype: object)

In [12]:
# function for calculating weighted average of D_Score and E_Score to predict the individual score of Paris Olympics
def calculate_weighted_average_Paris(group, score_type):
    weighted_averages = {}

    for index, row in group.iterrows():
      prediction_date = pd.Timestamp("2024-7-30")
      if group.shape[0] > 1:
        group_deleted = group.drop(index)
        #continue
      else:
        group_deleted = group

      # calculate differences between prediction dates and each competition date(in days)
      time_diff = (prediction_date - group_deleted['Date']).dt.days

      # time decay weights
      weights = np.exp(-lambda_decay * abs(time_diff))
    
      # weighted average
      weighted_average = np.average(group_deleted[score_type], weights=weights)

      weighted_averages[row["Competition_Round"]] = weighted_average #Competition대신 해당 행을 나타낼 수 있는 것 필요

    return weighted_averages

In [13]:
# recent score
recent_rec = data_cleaned.sort_values(by='Date').groupby(['FirstName', 'LastName','Round', 'Apparatus']).last().reset_index()
  

In [14]:
# Apply the updated function to each group of Competition and Apparatus
rank_contributions_v3 = recent_rec.groupby(['Competition', 'Apparatus']).apply(calculate_rank_contribution_v2)

# Drop the level added by groupby and sort by the original index
rank_contributions_v3.reset_index(level=[0, 1], drop=True, inplace=True)
rank_contributions_v3.sort_index(inplace=True)

# Update the RankContribution column in the original data
recent_rec['RankContribution'] = rank_contributions_v3['RankContribution']

# Recalculate the total rank contribution for each athlete
recent_rec['TotalRankContribution'] = recent_rec.groupby(['FirstName', 'LastName','Apparatus'])['RankContribution'].transform('sum')

# Display the updated data
recent_rec.head()

FirstName   LastName Round Apparatus Gender Country       Date  \
0         .  ABIYURAFI  qual        HB      m     INA 2023-03-01   
1         .  ABIYURAFI  qual        PB      m     INA 2023-03-01   
2         .  ABIYURAFI  qual        PH      m     INA 2023-03-01   
3      Abby     MEADOW  qual        BB      w     GBR 2022-03-24   
4      Abby     MEADOW  qual        FX      w     GBR 2022-03-24   

                             Competition            Location  Rank  ...  \
0                    2023 Doha World Cup         Doha, Qatar  26.0  ...   
1                    2023 Doha World Cup         Doha, Qatar  29.0  ...   
2                    2023 Doha World Cup         Doha, Qatar  39.0  ...   
3  2022 British Gymnastics Championships  Liverpool, England  61.0  ...   
4  2022 British Gymnastics Championships  Liverpool, England  49.0  ...   

   Home_advantage                         id    full_name      Beta  \
0               0      Indonesia_._abiyurafi  ._abiyurafi       NaN   
1               0      Indonesia_._abiyurafi  ._abiyurafi       NaN   
2               0      Indonesia_._abiyurafi  ._abiyurafi       NaN   
3               0  Great Britain_abby_meadow  abby_meadow  0.015165   
4               0  Great Britain_abby_meadow  abby_meadow  0.015165   

  Adjusted_Rank Adjusted_DRank Adjusted_ERank  RankContribution  \
0            27             16             28          0.000000   
1            29             26             29          1.434000   
2            39             39             38          0.167000   
3            60             50             61          1.030833   
4            80             88             62          1.548739   

  TotalRankContribution                          Competition_Round  
0              0.000000                    2023 Doha World Cupqual  
1              1.434000                    2023 Doha World Cupqual  
2              0.167000                    2023 Doha World Cupqual  
3              1.030833  2022 British Gymnastics Championshipsqual  
4              1.548739  2022 British Gymnastics Championshipsqual  

[5 rows x 27 columns]

In [15]:
# weighted average to predict the score of Paris Olympics
weighted_averages_d_Paris = recent_rec.groupby(['Athlete','Apparatus']).apply(lambda x: calculate_weighted_average_Paris(x, 'D_Score'))
weighted_averages_e_Paris = recent_rec.groupby(['Athlete','Apparatus']).apply(lambda x: calculate_weighted_average_Paris(x, 'E_Score'))
weighted_averages_rank_Paris = recent_rec.groupby(['Athlete','Apparatus']).apply(lambda x: calculate_weighted_average_Paris(x, 'Adjusted_Rank'))
weighted_averages_Drank_Paris = recent_rec.groupby(['Athlete','Apparatus']).apply(lambda x: calculate_weighted_average_Paris(x, 'Adjusted_DRank'))
weighted_averages_Erank_Paris = recent_rec.groupby(['Athlete','Apparatus']).apply(lambda x: calculate_weighted_average_Paris(x, 'Adjusted_ERank'))
weighted_averages_Rankcontr_Paris = recent_rec.groupby(['Athlete','Apparatus']).apply(lambda x: calculate_weighted_average_Paris(x, 'RankContribution'))
weighted_averages_d_Paris.head(), weighted_averages_e_Paris.head()

(Athlete      Apparatus
 AAS Fredrik  FX           {'2023 FISU World University Gamesqual': 4.0}
              HB           {'2023 FISU World University Gamesqual': 4.6}
              PB           {'2023 FISU World University Gamesqual': 4.4}
              PH           {'2023 FISU World University Gamesqual': 4.4}
              SR           {'2023 FISU World University Gamesqual': 3.5}
 dtype: object,
 Athlete      Apparatus
 AAS Fredrik  FX           {'2023 FISU World University Gamesqual': 8.566}
              HB           {'2023 FISU World University Gamesqual': 8.166}
              PB           {'2023 FISU World University Gamesqual': 8.066}
              PH           {'2023 FISU World University Gamesqual': 7.266}
              SR           {'2023 FISU World University Gamesqual': 8.366}
 dtype: object)

In [16]:
# data preparation for D_Score model
data_cleaned = data_cleaned.merge(weighted_averages_d.rename('Weighted_Average_D'), on=['Athlete','Apparatus'])
# data preparation for E_Score model
data_cleaned = data_cleaned.merge(weighted_averages_e.rename('Weighted_Average_E'), on=['Athlete','Apparatus'])
data_cleaned = data_cleaned.merge(weighted_averages_rank.rename('Weighted_Average_Rank'), on=['Athlete','Apparatus'])
data_cleaned = data_cleaned.merge(weighted_averages_Drank.rename('Weighted_Average_DRank'), on=['Athlete','Apparatus'])
data_cleaned = data_cleaned.merge(weighted_averages_Erank.rename('Weighted_Average_ERank'), on=['Athlete','Apparatus'])
data_cleaned = data_cleaned.merge(weighted_averages_Rankcontr.rename('Weighted_Average_Rankcon'), on=['Athlete','Apparatus'])

In [17]:
for i in range(data_cleaned.shape[0]):
  data_cleaned.at[i, "num_data"] = len(data_cleaned.at[i, "Weighted_Average_D"])
  data_cleaned.at[i, "Weighted_Average_D"] = data_cleaned.at[i, "Weighted_Average_D"][data_cleaned.at[i, "Competition_Round"]]
  data_cleaned.at[i, "Weighted_Average_E"] = data_cleaned.at[i, "Weighted_Average_E"][data_cleaned.at[i, "Competition_Round"]]
  data_cleaned.at[i, "Weighted_Average_Rank"] = data_cleaned.at[i, "Weighted_Average_Rank"][data_cleaned.at[i, "Competition_Round"]]
  data_cleaned.at[i, "Weighted_Average_DRank"] = data_cleaned.at[i, "Weighted_Average_DRank"][data_cleaned.at[i, "Competition_Round"]]
  data_cleaned.at[i, "Weighted_Average_ERank"] = data_cleaned.at[i, "Weighted_Average_ERank"][data_cleaned.at[i, "Competition_Round"]]
  data_cleaned.at[i, "Weighted_Average_Rankcon"] = data_cleaned.at[i, "Weighted_Average_Rankcon"][data_cleaned.at[i, "Competition_Round"]]

In [18]:
data_cleaned["Weighted_Average_D"] = data_cleaned["Weighted_Average_D"].astype(float)
data_cleaned["Weighted_Average_E"] = data_cleaned["Weighted_Average_E"].astype(float)
data_cleaned["Weighted_Average_Rank"] = data_cleaned["Weighted_Average_Rank"].astype(float)
data_cleaned["Weighted_Average_DRank"] = data_cleaned["Weighted_Average_DRank"].astype(float)
data_cleaned["Weighted_Average_ERank"] = data_cleaned["Weighted_Average_ERank"].astype(float)
data_cleaned["Weighted_Average_Rankcon"] = data_cleaned["Weighted_Average_Rankcon"].astype(float)

In [19]:

recent_rec = recent_rec.merge(weighted_averages_d_Paris.rename('Weighted_Average_D'), on=['Athlete','Apparatus'])
recent_rec = recent_rec.merge(weighted_averages_e_Paris.rename('Weighted_Average_E'), on=['Athlete','Apparatus'])
recent_rec = recent_rec.merge(weighted_averages_rank_Paris.rename('Weighted_Average_Rank'), on=['Athlete','Apparatus'])
recent_rec = recent_rec.merge(weighted_averages_Drank_Paris.rename('Weighted_Average_DRank'), on=['Athlete','Apparatus'])
recent_rec = recent_rec.merge(weighted_averages_Erank_Paris.rename('Weighted_Average_ERank'), on=['Athlete','Apparatus'])
recent_rec = recent_rec.merge(weighted_averages_Rankcontr_Paris.rename('Weighted_Average_Rankcon'), on=['Athlete','Apparatus'])

In [20]:
for i in range(recent_rec.shape[0]):
  recent_rec.at[i, "num_data"] = len(recent_rec.at[i, "Weighted_Average_D"])
  recent_rec.at[i, "Weighted_Average_D"] = recent_rec.at[i, "Weighted_Average_D"][recent_rec.at[i, "Competition_Round"]]
  recent_rec.at[i, "Weighted_Average_E"] = recent_rec.at[i, "Weighted_Average_E"][recent_rec.at[i, "Competition_Round"]]
  recent_rec.at[i, "Weighted_Average_Rank"] = recent_rec.at[i, "Weighted_Average_Rank"][recent_rec.at[i, "Competition_Round"]]
  recent_rec.at[i, "Weighted_Average_DRank"] = recent_rec.at[i, "Weighted_Average_DRank"][recent_rec.at[i, "Competition_Round"]]
  recent_rec.at[i, "Weighted_Average_ERank"] = recent_rec.at[i, "Weighted_Average_ERank"][recent_rec.at[i, "Competition_Round"]]
  recent_rec.at[i, "Weighted_Average_Rankcon"] = recent_rec.at[i, "Weighted_Average_Rankcon"][recent_rec.at[i, "Competition_Round"]]

In [21]:
recent_rec["Weighted_Average_D"] = recent_rec["Weighted_Average_D"].astype(float)
recent_rec["Weighted_Average_E"] = recent_rec["Weighted_Average_E"].astype(float)
recent_rec["Weighted_Average_Rank"] = recent_rec["Weighted_Average_Rank"].astype(float)
recent_rec["Weighted_Average_DRank"] = recent_rec["Weighted_Average_DRank"].astype(float)
recent_rec["Weighted_Average_ERank"] = recent_rec["Weighted_Average_ERank"].astype(float)
recent_rec["Weighted_Average_Rankcon"] = recent_rec["Weighted_Average_Rankcon"].astype(float)

# Score prediction(XGBoost)
 


We utilized the XGBoost model to predict scores for athletes in the Paris Olympics. The D Score exhibits relatively low variability for each athlete, depending on the attempted skills. In contrast, the E Score reflects how well an athlete performed in the actual competition, resulting in significantly greater variability. Due to these distinct characteristics of D Score and E Score, we trained separate models for each.

The XGBoost model was selected because it showed the best performance among Randomforest, Multi Layer Perceptron method.

We included the following variables for score prediction: Gender, Country, Apparatus, Round, Weighted_Average_D, RankContribution, Host, Beta.

We then split the data into training and testing sets for both D Score and E Score, with random selection.

The prediction results showed a relatively high accuracy with a rmse about 0.3 f0r D Score and rmse about 0.60 for E Score.

In [52]:
# feature selection
features_d = ['Gender', 'Country', 'Apparatus','Competition','Round','Weighted_Average_D',"Weighted_Average_DRank","Weighted_Average_Rank",'Weighted_Average_Rankcon','Home_advantage','Beta']

# Encode categorical data
label_encoders = {}
for feature in features_d[:5]:
    label_encoders[feature] = LabelEncoder()
    data_cleaned[feature] = label_encoders[feature].fit_transform(data_cleaned[feature])

# Prepare training data and target (goal) variable
X_d = data_cleaned[features_d]
y_d_score = data_cleaned['D_Score']


# Split into training and test sets (D_Score)
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_d, y_d_score, test_size=0.2, random_state=42)

# XGBoost model
model_d = xgb.XGBRegressor(objective ='reg:squarederror')


# model train
model_d.fit(X_train_d, y_train_d)

y_pred_d = model_d.predict(X_test_d)
rmse_d = np.sqrt(mean_squared_error(y_test_d, y_pred_d))
rmse_d

0.3002342413963878

In [53]:
# feature selection
features_e = ['Gender', 'Country','Apparatus','Competition','Round','Weighted_Average_E',"Weighted_Average_ERank","Weighted_Average_Rankcon",'Home_advantage','Beta']

# Encode categorical data
label_encoders = {}
for feature in features_e[:5]:
    label_encoders[feature] = LabelEncoder()
    data_cleaned[feature] = label_encoders[feature].fit_transform(data_cleaned[feature])

# Prepare training data and target (goal) variable
X_e = data_cleaned[features_e]
y_e_score = data_cleaned['E_Score']


# Split into training and test sets (E_Score)
X_train_e, X_test_e, y_train_e, y_test_e = train_test_split(X_e, y_e_score, test_size=0.2, random_state=42)

# XGBoost model
model_e = xgb.XGBRegressor(objective ='reg:squarederror')


# model train
model_e.fit(X_train_e, y_train_e)

y_pred_e = model_e.predict(X_test_e)
rmse_e = np.sqrt(mean_squared_error(y_test_e, y_pred_e))
rmse_e

0.6597724599956857

In [43]:
#select USA players
label_encoders = {}
for feature in features_e[:5]:
    label_encoders[feature] = LabelEncoder()
    recent_rec[feature] = label_encoders[feature].fit_transform(recent_rec[feature])


US_gym = recent_rec[recent_rec["Country"]==97]

#fit the model
US_pred_e = model_e.predict(US_gym[features_e])
US_pred_d = model_d.predict(US_gym[features_d])
US_pred = US_pred_e + US_pred_d

for feature in label_encoders:
    US_gym[feature] = label_encoders[feature].inverse_transform(US_gym[feature])


C:\Users\sbjbo\AppData\Local\Temp\ipykernel_37872\3388516798.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  US_gym[feature] = label_encoders[feature].inverse_transform(US_gym[feature])
C:\Users\sbjbo\AppData\Local\Temp\ipykernel_37872\3388516798.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  US_gym[feature] = label_encoders[feature].inverse_transform(US_gym[feature])
C:\Users\sbjbo\AppData\Local\Temp\ipykernel_37872\3388516798.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [44]:
#create prediction score column
US_gym["Score_Prediction"]=US_pred

C:\Users\sbjbo\AppData\Local\Temp\ipykernel_37872\483226364.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  US_gym["Score_Prediction"]=US_pred


In [45]:
US_gym = US_gym[US_gym["Round"]!="qual"]

In [ ]:
US_gym.to_csv("USA_gym.csv")